In [9]:
import xarray as xr
import rioxarray
import numpy as np
from pathlib import Path
from datetime import datetime, timedelta
from cftime import date2num
from tqdm import tqdm
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

## Download SWOT tif files

In [89]:
# Download SWOT data
def downlad_swot_data(
        url:str,
        dst:str,
        download:bool) -> list:
    """ 
    Download SWOT tif files from the http
    """
    urls = []
    url = url.replace(" ","%20")
    req = Request(url)
    a = urlopen(req).read()
    soup = BeautifulSoup(a, 'html.parser')
    x = (soup.find_all('a'))
    for i in tqdm(x):
        file_name = i.extract().get_text()
        url_new = url + file_name
        url_new = url_new.replace(" ","%20")
        if url_new.endswith('tif'):
            f_name = Path(url_new).name
            urls.append(url_new)
            if download:
                urlretrieve(url_new, dst / f_name)
    return urls

src = "https://data.aviso.altimetry.fr/aviso-gateway/data/swot-adac/norwegiansea-lofotenbasin/"
dst = Path('dev/swot_data/lofoten/')
url_list = downlad_swot_data(src, dst, True)

 32%|███▏      | 94/293 [01:14<02:41,  1.23it/s]

## Convert SWOT data to the IDF format

In [10]:
IDF_TIME_UNITS = 'days since 1970-01-01 00:00:00.000000Z'

IDF_BANDS = {
    'lat_gcp': {
        'dimensions': ('row_gcp', 'cell_gcp'),
        'datatype': 'f4',
        'metadata': {
            'long_name': 'ground control points latitude',
            'standard_name': 'latitude',
            'units': 'degrees_north',
            'axis': 'Y',
            'comment': 'geographical coordinates, WGS84 projection'}},
    'lon_gcp': {
        'dimensions': ('row_gcp', 'cell_gcp'),
        'datatype': 'f4',
        'metadata': {
            'long_name': 'ground control points longitude',
            'standard_name': 'longitude',
            'units': 'degrees_east',
            'axis': 'X',
            'comment': 'geographical coordinates, WGS84 projection'}},
    'index_row_gcp': {
        'dimensions': ('row_gcp'),
        'datatype': np.int32,
        'metadata': {
            'long_name': 'index of ground control points in row dimension',
            'comment': 'index goes from 0 (start of first pixel) to dimension value (end of last pixel)'}},
    'index_cell_gcp': {
        'dimensions': ('cell_gcp'),
        'datatype': np.int32,
        'metadata': {
            'long_name': 'index of ground control points in cell dimension',
            'comment': 'index goes from 0 (start of first pixel) to dimension value (end of last pixel)'}},
}

In [15]:

def gen_gcp_ids(size, step):
    # NOTE: last gcp id should be the last row or cell
    gcp_ids = list(np.arange(0, size, step)) + [size - 1]
    return gcp_ids

# Define the path with swot .tif files
src_root = Path('dev/swot_data/lofoten/')
dst_root = src_root / 'idf'
dst_root.mkdir(exist_ok=True)

for uri in tqdm(list(src_root.glob('*sig0.tif'))):
    s0_uri = Path(uri)
    ssh_uri = s0_uri.parent / s0_uri.name.replace('sig0', 'ssh')
    
    if not ssh_uri.exists() or not s0_uri.exists():
        continue

    swot_s0 = rioxarray.open_rasterio(s0_uri)
    swot_ssh = rioxarray.open_rasterio(ssh_uri)

    
    lon_grd, lat_grd  = np.meshgrid(swot_s0.x.data, swot_s0.y.data)

    s0_data = swot_s0[0,:,:].data
    s0_data[swot_s0[3,:,:].data == 0] = 25


    ssh_data = swot_ssh[0,:,:].data
    ssh_data[swot_ssh[3,:,:].data == 0] = 25

    out_ds = xr.Dataset(
        data_vars=dict(
            sigma0=(['time', 'row', 'cell'], [s0_data], {'_FillValue': 25}),
            ssh=(['time', 'row', 'cell'], [ssh_data], {'_FillValue': 25})),
        coords=dict(
            lat=(['row', 'cell'], lat_grd),
            lon=(['row', 'cell'], lon_grd)))

    if s0_uri.stem.split('_')[-2] == 'p005':
        time_start = datetime.strptime(s0_uri.stem.split('_')[1], '%Y%m%d') + timedelta(hours=1)
        idf_filename = '_'.join([s0_uri.stem, f'{s0_uri.stem.split("_")[1]}01Z_idf_00.nc'])
    elif s0_uri.stem.split('_')[-2] == 'p014':
        time_start = datetime.strptime(s0_uri.stem.split('_')[1], '%Y%m%d') + timedelta(hours=6)
        idf_filename = '_'.join([s0_uri.stem, f'{s0_uri.stem.split("_")[1]}06Z_idf_00.nc'])
    else:
        raise ValueError

    time_end = time_start  + timedelta(hours=1)

    out_ds['time'] = xr.DataArray(
            [date2num(time_start, IDF_TIME_UNITS)], dims=['time'], 
            attrs=dict(units=IDF_TIME_UNITS, long_name='time', standard_name='time', calendar='gregorian'))

    out_ds = out_ds.assign_attrs({
            'idf_version': 1.0,
            'idf_subsampling_factor': 0,
            'idf_spatial_resolution': 1000,
            'idf_spatial_resolution_units': 'm', 
            'idf_granule_id': idf_filename.replace('.nc', ''),
            'time_coverage_start': f'{time_start:%Y-%m-%dT%H:%M:%S}.000000Z',
            'time_coverage_end': f'{time_end:%Y-%m-%dT%H:%M:%S}.000000Z'
        })

        # Add idf GCP variables
    gcp_ids_row = gen_gcp_ids(out_ds.row.size, 10)
    gcp_ids_cell = gen_gcp_ids(out_ds.cell.size, 10)
    out_ds = out_ds.assign(
            lat_gcp=(['row_gcp', 'cell_gcp'], out_ds['lat'].data[gcp_ids_row, :][:, gcp_ids_cell], IDF_BANDS['lat_gcp']['metadata']),
            lon_gcp=(['row_gcp', 'cell_gcp'], out_ds['lon'].data[gcp_ids_row, :][:, gcp_ids_cell], IDF_BANDS['lon_gcp']['metadata']),
            index_row_gcp=(['row_gcp'], gcp_ids_row, IDF_BANDS['index_row_gcp']['metadata']),
            index_cell_gcp=(['cell_gcp'], gcp_ids_cell, IDF_BANDS['index_cell_gcp']['metadata']))

    out_ds = out_ds.drop_vars(['lat', 'lon'])

    out_ds.to_netcdf(dst_root / idf_filename)


100%|██████████| 113/113 [00:25<00:00,  4.47it/s]
